In [821]:
pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/124.9 MB 3.4 MB/s eta 0:00:38
   - -------------------------------------- 3.1/124.9 MB 9.7 MB/s eta 0:00:13
   -- ------------------------------------- 9.2/124.9 MB 17.8 MB/s eta 0:00:07
   ---- ----------------------------------- 13.4/124.9 MB 18.3 MB/s eta 0:00:07
   ------ --------------------------------- 21.2/124.9 MB 24.9 MB/s eta 0:00:05
   --------- ------------------------------ 30.9/124.9 MB 27.7 MB/s eta 0:00:04
   ------------ --------------------------- 39.6/124.9 MB 29.6 MB/s eta 0:00:03
   --------------- ------------------------ 48.2/124.9 MB 31.3 MB/s eta 0:00:03
   ------------------ --------------------- 56.4/124.9 MB 32.4 MB/s eta 0:00:03
   -------------------- ------------------- 64.0/124.9 MB 32.4 MB/s eta 0:00:02
   ----------------------- ---------------- 73.7/124.9 MB 33.6 

In [825]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score , classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [786]:
data = pd.read_csv('Quiz_Question_Data_2.csv')

In [788]:
skill_mapping = {'Beginner': 0, 'Intermediate': 1, 'Advanced': 2}
data['Skill_Level_encoded'] = data['Skill Level'].map(skill_mapping)

# Multiply the 'Time Taken (QNS/Minutes)' column by 60 to convert to seconds
data['Time Taken (QNS/Seconds)'] = (data['Time Taken (QNS/Minutes)'] * 60).round(4)

# Drop the original 'Time Taken (QNS/Minutes)' column
data.drop(columns=['Time Taken (QNS/Minutes)'], inplace=True)

# Optionally, save the modified data back to a new Excel file
data.to_csv('modified_file.csv', index=False)

In [790]:
import pandas as pd

# Check for missing values in the entire dataset
missing_data = data.isnull().sum()
print(missing_data)

# Check if there are any missing values
if data.isnull().values.any():
    print("There are missing values in the dataset.")
else:
    print("No missing values in the dataset.")

data = data.dropna()

Question Difficulty         0
Topic Difficulty            0
Score                       0
Skill Level                 0
Skill_Level_encoded         0
Time Taken (QNS/Seconds)    0
dtype: int64
No missing values in the dataset.


In [809]:
data = pd.read_csv('modified_file.csv')
X = data[['Time Taken (QNS/Seconds)', 'Question Difficulty', 'Topic Difficulty']]
Y = data['Skill_Level_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf = RandomForestClassifier(random_state=42) 
param_grid = {
    'n_estimators': [350, 380, 400],
    'max_depth': [10, 15, 20],
    'max_features': ['log2','None,'sqrt'],
    'min_samples_split': [15,20,25],
    'min_samples_leaf': [3, 5, 8],
    'bootstrap': [True,False]
}
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=100, random_state=42, n_jobs=-1, verbose=2)

random_search.fit(X_train_scaled, y_train)
print("Best parameters:", random_search.best_params_)

model = random_search.best_estimator_
y_pred = model.predict(X_test_scaled)

Training set size: (8000, 3)
Testing set size: (2000, 3)
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters: {'n_estimators': 350, 'min_samples_split': 30, 'min_samples_leaf': 3, 'max_features': 'log2', 'max_depth': 10, 'bootstrap': True}


In [813]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 72.25%
